In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set(color_codes=True)

In [26]:
url_train="https://raw.githubusercontent.com/Abhijeet-Katiyar/Big-MArt-Sales-prediction/master/Train_UWu5bXk.csv"
url_test="https://raw.githubusercontent.com/Abhijeet-Katiyar/Big-MArt-Sales-prediction/master/Test_u94Q5KV.csv"

Train_data=pd.read_csv(url_train)
Test_data=pd.read_csv(url_test)

In [27]:
Data=Train_data.append(Test_data,sort=False)

In [28]:
Data['Item_Fat_Content'] = Data['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})
Data.groupby('Item_Identifier')['Item_Weight'].mean().head(5)
for i in Data.groupby('Item_Identifier')['Item_Weight'].mean().index:
    Data.loc[Data.loc[:,'Item_Identifier']==i,'Item_Weight']=Data.groupby('Item_Identifier')['Item_Weight'].mean()[i]

In [29]:
Data['Outlet_Type'].value_counts()

Supermarket Type1    9294
Grocery Store        1805
Supermarket Type3    1559
Supermarket Type2    1546
Name: Outlet_Type, dtype: int64

In [30]:
print(Data.Outlet_Size[Data['Outlet_Type']=='Grocery Store'].value_counts())
print(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type1'].value_counts())
print(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type2'].value_counts())

Small    880
Name: Outlet_Size, dtype: int64
Small     3100
High      1553
Medium    1550
Name: Outlet_Size, dtype: int64
Medium    1546
Name: Outlet_Size, dtype: int64


In [31]:
#Data['Outlet_Size'].fillna(Data['Outlet_Size'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Grocery Store'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type1'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type2'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type3'].mode()[0],inplace=True)

In [32]:
for i in Data.groupby('Item_Identifier')['Item_Visibility'].mean().index:
    Data.loc[Data.loc[:,'Item_Identifier']==i,'Item_Visibility']=Data.groupby('Item_Identifier')['Item_Visibility'].mean()[i]

In [33]:
Data['Outlet_Establishment_Year']=2013-Data['Outlet_Establishment_Year']

In [34]:
Data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

In [35]:
Train_data=Data.dropna()

In [36]:
Test_Data=Data[Data['Item_Outlet_Sales'].isnull()]
Test_Data.drop('Item_Outlet_Sales',axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
from sklearn.preprocessing import LabelEncoder

In [38]:
categorical_list=['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Size','Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year']

In [39]:
le = LabelEncoder()
for i in categorical_list:
    Train_data[i]=le.fit_transform(Train_data[i])
    Train_data[i]=Train_data[i].astype('category')
    Test_Data[i]=le.fit_transform(Test_Data[i])
    Test_Data[i]=Test_Data[i].astype('category')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [40]:
print("Train Data")
print(Train_data.head())
print("Test Data")
print(Test_Data.head())

Train Data
  Item_Identifier  Item_Weight  ... Outlet_Type  Item_Outlet_Sales
0           FDA15         9.30  ...           1          3735.1380
1           DRC01         5.92  ...           2           443.4228
2           FDN15        17.50  ...           1          2097.2700
3           FDX07        19.20  ...           0           732.3800
4           NCD19         8.93  ...           1           994.7052

[5 rows x 12 columns]
Test Data
  Item_Identifier  Item_Weight  ... Outlet_Location_Type  Outlet_Type
0           FDW58       20.750  ...                    0            1
1           FDW14        8.300  ...                    1            1
2           NCN55       14.600  ...                    2            0
3           FDQ58        7.315  ...                    1            1
4           FDY38       13.600  ...                    2            3

[5 rows x 11 columns]


In [41]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [59]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=9, kernel_initializer='Orthogonal', activation='relu'))  
    model.add(Dense(40, kernel_initializer='Orthogonal', activation='relu'))  
    model.add(Dense(50, kernel_initializer='Orthogonal', activation='relu'))
    model.add(Dense(30, kernel_initializer='Orthogonal', activation='relu'))
    model.add(Dense(20, kernel_initializer='Orthogonal', activation='relu'))  
    model.add(Dense(10,activation='relu',kernel_initializer='Orthogonal'))  
    model.add(Dense(1, kernel_initializer='Orthogonal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [60]:
seed = 7
np.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=150, batch_size=12, verbose=0)

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
 

In [62]:
estimator.fit(X_train,y_train, batch_size = 12, epochs = 50)
predictions=estimator.predict(X_test)

from sklearn.metrics import r2_score
print("Score: ",r2_score(y_test,predictions))

Score:  0.6035092936819981


In [44]:
X=np.asarray((Train_data.drop(['Item_Outlet_Sales','Outlet_Identifier','Item_Identifier'],axis=1))).astype(np.float32)
Y=np.asarray(Train_data['Item_Outlet_Sales']).astype(np.float32)
estimator.fit(X,Y, batch_size = 12, epochs = 50)

In [49]:
train=np.asarray(Train_data.drop(['Item_Outlet_Sales','Outlet_Identifier','Item_Identifier'],axis=1)).astype(np.float32)
predictions=np.asarray(Train_data['Item_Outlet_Sales']).astype(np.float32)
out=[]
for i in range(len(Test_Data)):
    estimator.fit(np.asarray(Train_data.drop(['Item_Outlet_Sales','Outlet_Identifier','Item_Identifier'],axis=1)).astype(np.float32),np.asarray(Train_data['Item_Outlet_Sales']).astype(np.float32), batch_size = 12, epochs = 50)
    Output=estimator.predict(np.asarray(Test_Data.drop(['Item_Identifier','Outlet_Identifier'],axis=1)[Test_Data.index==i]).astype(np.float32))
    out.append(Output)
    train.append(Test_Data[Test_Data.index==i])
    predictions.append(pd.Series(Output))

AttributeError: ignored

In [ ]:
out